In [0]:
%sql
CREATE CATALOG IF NOT EXISTS dev
MANAGED LOCATION 'abfss://unity-catalog-storage@dbstoragehakrjbhbggnca.dfs.core.windows.net/2598701134546467';

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS dev.demo_db;


In [0]:
%sql
DROP DATABASE IF EXISTS dev.default CASCADE;

In [0]:
flight_schema_ddl = """FL_DATE DATE, OP_CARRIER STRING, OP_CARRIER_FL_NUM INT, ORIGIN STRING, 
          ORIGIN_CITY_NAME STRING, DEST STRING, DEST_CITY_NAME STRING, CRS_DEP_TIME INT, DEP_TIME INT, 
          WHEELS_ON INT, TAXI_IN INT, CRS_ARR_TIME INT, ARR_TIME INT, CANCELLED STRING, DISTANCE INT"""

flight_time_dt = spark.read.format("csv").schema(flight_schema_ddl).option("dateFormat", "M/d/y").load("abfss://dbfs-container@azuredatabrickstorages.dfs.core.windows.net/external/flight-time.json")


The cell above is a dataframe to create a schema and load data from the external storage (Adlsgen2) which is in json file before.
then read the data in a csv foramt , then display the content of the file 

NOTE : the content is not save yet you are just reading or viewing the file content , because json file content are not structure like csv file 

In [0]:
display (flight_time_dt)

In [0]:
%sql
CREATE TABLE IF NOT EXISTS dev.demo_db.flight_time_tbl (
    FL_DATE DATE, 
    OP_CARRIER STRING, 
    OP_CARRIER_FL_NUM INT, 
    ORIGIN STRING, 
    ORIGIN_CITY_NAME STRING, 
    DEST STRING, 
    DEST_CITY_NAME STRING, 
    CRS_DEP_TIME INT, 
    DEP_TIME INT, 
    WHEELS_ON INT, 
    TAXI_IN INT, 
    CRS_ARR_TIME INT, 
    ARR_TIME INT, 
    CANCELLED STRING, 
    DISTANCE INT
) USING DELTA

In [0]:
flight_time_dt.write.format("delta").mode("append").saveAsTable("dev.demo_db.flight_time_tbl")

In [0]:
flight_time_tbl = spark.read.table("dev.demo_db.flight_time_tbl")
display(flight_time_tbl)

In [0]:
%sql
select * from dev.demo_db.flight_time_tbl

Save delta table in External Location (ADLSgen2)

In [0]:
(flight_time_dt.coalesce(1).write.format("delta").mode("overwrite").save("abfss://dbfs-container@azuredatabrickstorages.dfs.core.windows.net/external/flight-time/Folex"))

In [0]:
LOAD THE PEOPLE JSON FILE INTO THE DATAFRAME AND REVIEW/ READ

In [0]:
people_df = (spark.read.format("json").load("abfss://dbfs-container@azuredatabrickstorages.dfs.core.windows.net/external/people.json"))
display (people_df)


WRITE / SAVE THE PEOPLE DATA INTO THE DEV CATALOG USING PYTHON SCRIPT

In [0]:
people_df.write.format("delta").mode("overwrite").saveAsTable("dev.demo_db.people")

## read table 
people = spark.read.table("dev.demo_db.people")
display (people)


LET LEARN HOW TO DELETE RECORD FROM THE CATALOG

In [0]:
%sql

DELETE FROM DEV.DEMO_DB.PEOPLE
WHERE ID = 102;

select * from dev.demo_db.people

LET LEARN HOW TO UPDATE USING SQL SYNTANX

In [0]:
%sql
update dev.demo_db.people
set dob = '1985-05-25'
where dob = '1975-05-25';

select * from dev.demo_db.people

LET LOOK AT TIME TRAVEL
WHICH HELP US TO SEE HISTORY OF WHAT WE HAVE BEEN DOING IN THE PAST AND WE CAN REVERT/RESTORE TO PREVIOUS VERSION 

In [0]:
%sql

describe history dev.demo_db.people;



In [0]:
%sql
select * from dev.demo_db.people version as of 1

In [0]:
%sql
select * from dev.demo_db.people 
timestamp as of '2025-06-24T04:18:27.000+00:00'

DELET THE DELTA TABLE BY MISTAKE 
let assusume you mistakenly delete the entire record in your table and you want to restore back to previous record 

In [0]:
%sql

delete from dev.demo_db.people;

select * from dev.demo_db.people

ROLL BACK TO RESTORE TO PREVIOUS HISTORY

In [0]:
%sql
describe history dev.demo_db.people

In [0]:
%sql
restore table dev.demo_db.people to timestamp as of '2025-06-24T04:47:32.000+00:00';

select * from dev.demo_db.people